## TRDE703 Atelier Intégration des Données

In [ ]:
import sys
import os
from pathlib import Path

# --- 1. D'ABORD : ON CONFIGURE LES CHEMINS (Le "Hack") ---
# On doit le faire AVANT d'importer 'etl.*' sinon Python ne trouve pas le dossier.

current_dir = Path(os.getcwd())
# Si le notebook est dans 'etl/', la racine est le dossier parent
project_root = current_dir.parent if current_dir.name == "etl" else current_dir

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print(f"✅ Racine ajoutée au path : {project_root}")

# --- 2. ENSUITE : ON PEUT IMPORTER TON CODE ---
from pyspark.sql import SparkSession
from etl.shared.config import SPARK_CONFIG  # Maintenant ça va marcher !

# --- 3. ENFIN : ON LANCE SPARK AVEC LE JAR ---
%load_ext autoreload
%autoreload 2

builder = SparkSession.builder
for key, val in SPARK_CONFIG.items():
    builder = builder.config(key, val)

spark = builder.getOrCreate()

print(f"✅ Session Spark créée avec le JAR : {SPARK_CONFIG.get('spark.jars')}")

In [ ]:
from etl.shared.config import SPARK_CONFIG, MYSQL_CONFIG
from pyspark.sql import SparkSession

print("⚙️ Configuration chargée avec succès.")

In [ ]:
# On construit le chemin proprement avec pathlib
# project_root est déjà défini dans la première cellule
json_filepath = str(project_root / "data" / "raw" / "openfoodfacts-products.jsonl")

print(f"📂 Fichier cible : {json_filepath}")

if os.path.exists(json_filepath):
    print("✅ Le fichier existe bien.")
else:
    print("❌ Fichier introuvable. Vérifie le dossier data/raw/")

### 💡 Un mot sur nos choix (et les consignes du TP)

Pourquoi s'embêter à écrire ce schéma manuellement ?

1.  **Respect de la consigne :** Le sujet est strict : *"Lecture JSON/CSV avec schéma explicite (pas d'inférence magique en prod)"*. Utiliser `inferSchema=True` nous ferait perdre des points.
2.  **Gestion de l'Historique (SCD2) :** Le sujet impose de gérer le *"SCD2 produit"*. Pour cela, nous avons impérativement besoin du timestamp brut (`last_modified_t` en `LongType`) pour comparer les versions à la seconde près.
3.  **Structure Imbriquée :** Le format JSONL groupe les nutriments dans un objet. Notre schéma reflète cette réalité (`StructType` imbriqué) pour éviter de créer 1000 colonnes plates inutiles.
4.  **Sécurité (`String`) :** Pour des champs instables comme `nova_group`, on lit en `String` pour éviter que Spark ne rejette la ligne en cas de format inattendu.

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, LongType, ArrayType

def get_jsonl_schema():
    """
    Schéma robuste pour l'ingestion JSONL.
    Gère les types imbriqués (nested) propres à MongoDB/JSON.
    """

    # Définition de la sous-structure pour les nutriments
    # (Permet de lire l'objet "nutriments": { ... } proprement)
    nutriments_schema = StructType([
        StructField("energy-kcal_100g", FloatType(), True),
        StructField("sugars_100g", FloatType(), True),
        StructField("salt_100g", FloatType(), True),
        StructField("sodium_100g", FloatType(), True),
        StructField("fiber_100g", FloatType(), True),
        StructField("proteins_100g", FloatType(), True)
    ])

    return StructType([
        # --- Identifiants & Métadonnées (Crucial pour SCD2) ---
        StructField("code", StringType(), True),            # La clé primaire métier (EAN)
        StructField("product_name", StringType(), True),
        StructField("last_modified_t", LongType(), True),   # Timestamp UNIX (versioning)
        StructField("created_t", LongType(), True),

        # --- Dimensions (Marques, Catégories...) ---
        StructField("brands", StringType(), True),
        StructField("categories", StringType(), True),
        StructField("countries_tags", ArrayType(StringType()), True), # Liste de pays ["en:france", "en:belgium"]

        # --- Qualité & Scores ---
        StructField("nutriscore_grade", StringType(), True),
        StructField("nova_group", IntegerType(), True),     # Souvent un entier dans le JSON
        StructField("ecoscore_grade", StringType(), True),

        # --- Mesures (Imbriquées) ---
        StructField("nutriments", nutriments_schema, True)  # L'objet imbriqué
    ])

print("✅ Schéma JSONL défini.")

In [ ]:
# Adapte le nom du fichier si nécessaire
input_file = "openfoodfacts-products.jsonl"
raw_path = str(project_root / "data" / "raw" / input_file)

print(f"📂 Préparation de la lecture : {raw_path}")

try:
    # 1. Lecture "Lazy" (Paresseuse)
    # Spark ne lit rien pour l'instant, il note juste le plan d'action.
    df_raw = spark.read \
        .schema(get_jsonl_schema()) \
        .json(raw_path)

    print("✅ Lecture configurée (Lazy). Le chargement réel se fera après le sampling.")


    count = df_raw.count()
    print(f"📊 Nombre de produits ingérés : {count:,}")

    df_raw.printSchema()

except Exception as e:
    print(f"❌ Erreur de lecture : {e}")

## 🧹 Étape 2 : Transformation "Silver" (Nettoyage & Typage)

Maintenant que les données brutes sont chargées, nous devons les rendre utilisables pour l'analyse et le SCD2.
Cette étape applique les règles de qualité demandées :

1.  **Typage Temporel :** Conversion des timestamps UNIX (`Long`) en vraies dates (`Timestamp`) pour `last_modified_t` et `created_t`.
2.  **Nettoyage Textuel :** Suppression des espaces superflus (`trim`) sur les codes-barres et noms.
3.  **Extraction des Nutriments :** Aplatissement de la structure imbriquée `nutriments` pour faciliter les requêtes SQL futures.
4.  **Gestion des Nulls :** Conversion sécurisée de `nova_group` (texte vers entier) et filtrage des produits sans code-barre.

In [ ]:
from pyspark.sql.functions import col, trim, from_unixtime, to_timestamp, when

print("⏳ Démarrage du nettoyage Silver avec Sampling...")

# --- CORRECTION 1 : ECHANTILLONNAGE (SAMPLING) ---
# On ne garde que 10% des données (environ 400k lignes) pour sauver ton disque dur.
# seed=42 permet d'avoir toujours les mêmes 10% si tu relances.
df_sampled = df_raw.sample(withReplacement=False, fraction=0.1, seed=42)

df_silver = df_sampled \
    .select(
        # --- 1. Nettoyage des Clés & Textes ---
        trim(col("code")).alias("code"),
        trim(col("product_name")).alias("product_name"),

        # --- 2. Gestion Temporelle ---
        from_unixtime(col("last_modified_t")).cast("timestamp").alias("last_modified_ts"),
        from_unixtime(col("created_t")).cast("timestamp").alias("created_ts"),

        # --- 3. Normalisation des Dimensions ---
        col("countries_tags"),
        trim(col("brands")).alias("brands"),
        trim(col("categories")).alias("categories"),

        # --- 4. Qualité & Scores ---
        trim(col("nutriscore_grade")).alias("nutriscore_grade"),
        trim(col("ecoscore_grade")).alias("ecoscore_grade"),
        col("nova_group").cast("integer").alias("nova_group"),

        # --- 5. Nutriments ---
        col("nutriments.energy-kcal_100g").alias("energy_kcal_100g"),
        col("nutriments.sugars_100g").alias("sugars_100g"),
        col("nutriments.salt_100g").alias("salt_100g"),
        col("nutriments.proteins_100g").alias("proteins_100g")
    ) \
    .filter(col("code").isNotNull()) \
    .filter(col("code") != "")

# On met en cache ce petit échantillon
df_silver.cache()

count = df_silver.count()
print(f"✅ Nettoyage terminé sur l'échantillon. Produits restants : {count:,}")
print("(C'est normal d'en avoir moins, on a pris 10% volontairement !)")

display(df_silver.select("code", "last_modified_ts", "product_name").limit(5))

## 🔐 Étape 3 : Fingerprinting (Préparation SCD2)

Pour gérer l'historique (SCD2) efficacement, nous ne pouvons pas comparer toutes les colonnes à chaque fois.
Nous allons générer un **Hash Technique (`row_hash`)** : une empreinte digitale unique basée sur les colonnes métier.

* **Stratégie :** On concatène toutes les colonnes importantes (Nom, Marque, Nutriscore, Sucre...) et on applique un hachage SHA-256.
* **Intérêt :** Si le hash change, cela signifie que le produit a été modifié. C'est ce qui déclenchera la création d'une nouvelle version dans le Datamart.

In [ ]:
from pyspark.sql.functions import sha2, concat_ws, col

print("⏳ Calcul du Hash (Fingerprint) pour chaque produit...")

# On sélectionne les colonnes qui doivent déclencher une nouvelle version si elles changent.
# Note d'Architecte : On NE met PAS 'last_modified_ts' dans le hash, car on veut détecter les changements de CONTENU métier.
columns_to_hash = [
    "product_name", "brands", "categories", "countries_tags",
    "nutriscore_grade", "nova_group", "ecoscore_grade",
    "energy_kcal_100g", "sugars_100g", "salt_100g", "proteins_100g"
]

df_hashed = df_silver.withColumn(
    "row_hash",
    sha2(concat_ws("||", *[col(c) for c in columns_to_hash]), 256)
)

print("✅ Hashing terminé.")
display(df_hashed.select("code", "product_name", "row_hash").limit(5))

## 🏭 Étape 4 : Initialisation du Datamart (DDL)

Avant de charger les données, nous devons créer la structure des tables dans MySQL.
Nous utilisons une connexion Python directe (hors Spark) pour définir précisément :
1.  **Les Clés Primaires (PK) :** `product_sk` (Auto-incrément) pour identifier unique une *version* de produit.
2.  **Les Index :** Sur `code` et `row_hash` pour que les recherches (Join/Upsert) soient instantanées.
3.  **Les Colonnes SCD2 :** `effective_from` (début), `effective_to` (fin), `is_current` (actif).

In [ ]:
import mysql.connector
from etl.shared.config import MYSQL_CONFIG

def init_datamart():
    print("⏳ Initialisation et Tuning MySQL...")

    conn = mysql.connector.connect(
        host="localhost",
        port=3306,
        user=MYSQL_CONFIG["user"],
        password=MYSQL_CONFIG["password"],
        database="openfoodfacts"
    )
    cursor = conn.cursor()

    # --- CORRECTION 2 : TUNING MYSQL (CRITIQUE) ---
    # On autorise des paquets jusqu'à 64 Mo pour éviter le "PacketTooBigException"
    cursor.execute("SET GLOBAL max_allowed_packet=67108864")
    print("   - Option MySQL 'max_allowed_packet' passée à 64MB.")

    # Suppression (Faits d'abord, Dimensions ensuite)
    cursor.execute("DROP TABLE IF EXISTS fact_nutrition_snapshot")
    cursor.execute("DROP TABLE IF EXISTS dim_product")

    # Création Dimension (avec TEXT pour les champs longs)
    product_ddl = """
    CREATE TABLE dim_product (
        product_sk INT AUTO_INCREMENT PRIMARY KEY,
        code VARCHAR(255) NOT NULL,

        product_name TEXT, -- TEXT car VARCHAR(500) est trop court pour certains produits
        brands TEXT,       -- TEXT car les listes de marques peuvent être immenses
        categories TEXT,

        row_hash CHAR(64) NOT NULL,
        effective_from DATETIME,
        effective_to DATETIME,
        is_current BOOLEAN,

        INDEX idx_code (code),
        INDEX idx_hash (row_hash)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    cursor.execute(product_ddl)
    print("   - Table 'dim_product' créée (avec TEXT).")

    # Création Faits
    fact_ddl = """
    CREATE TABLE fact_nutrition_snapshot (
        fact_sk INT AUTO_INCREMENT PRIMARY KEY,
        product_sk INT NOT NULL,
        date_sk INT NOT NULL,

        nutriscore_grade CHAR(1),
        ecoscore_grade CHAR(1),
        nova_group INT,
        energy_kcal_100g FLOAT,
        sugars_100g FLOAT,
        salt_100g FLOAT,
        proteins_100g FLOAT,

        FOREIGN KEY (product_sk) REFERENCES dim_product(product_sk)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    """
    cursor.execute(fact_ddl)
    print("   - Table 'fact_nutrition_snapshot' créée.")

    conn.close()
    print("✅ Datamart prêt.")

try:
    init_datamart()
except Exception as e:
    print(f"❌ Erreur MySQL : {e}")

## 🚚 Étape 5 : Chargement de la Dimension Produit (Initial Load)

Nous séparons les données en deux flux :
1.  **Dimension (`dim_product`) :** Contient les descriptions et l'historique.
2.  **Faits (`fact_nutrition_snapshot`) :** Contient les chiffres.

Ici, nous chargeons la dimension.
* **Transformation :** On ne garde que les colonnes descriptives.
* **Initialisation SCD2 :** Comme c'est le premier chargement, on fixe :
    * `effective_from` = La date de modification du produit (`last_modified_ts`).
    * `effective_to` = '9999-12-31' (Date infinie = produit actif).
    * `is_current` = True.
* **Écriture JDBC :** On pousse vers MySQL en mode `append`.

In [ ]:
from pyspark.sql.functions import lit
from etl.shared.config import MYSQL_CONFIG

print("⏳ Préparation de la dimension Produit...")

# Sélection finale
df_dim_product_init = df_hashed.select(
    col("code"),
    col("product_name"),
    col("brands"),
    col("categories"),
    col("row_hash"),
    col("last_modified_ts").alias("effective_from"),
    lit("9999-12-31 23:59:59").cast("timestamp").alias("effective_to"),
    lit(True).alias("is_current")
)

# Config JDBC optimisée
jdbc_url = MYSQL_CONFIG["url"]
jdbc_props = {
    "user": MYSQL_CONFIG["user"],
    "password": MYSQL_CONFIG["password"],
    "driver": MYSQL_CONFIG["driver"],

    # --- CORRECTION 3 : PETITS PAQUETS ---
    # On réduit à 1000 pour ménager le réseau et la mémoire
    "batchsize": "1000"
}

print("🚀 Écriture dans MySQL (dim_product)...")

try:
    df_dim_product_init.write \
        .jdbc(url=jdbc_url, table="dim_product", mode="append", properties=jdbc_props)

    print("✅ Chargement terminé avec succès !")

except Exception as e:
    print(f"❌ Erreur d'écriture : {e}")